In [ ]:
%pip install selenium
%pip install webdriver-manager
%pip install pandas
%pip install lxml
%pip install openpyxl
%pip install xlrd
%pip install yfinance

## Célula 1: Importações e Configurações Iniciais

In [2]:
# Célula 1: Importações e Configurações Iniciais
import os
import zipfile
import pandas as pd
import logging
import yfinance as yf
import math
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import StringIO
from datetime import datetime

# Configuração de logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Configuração das pastas
DOWNLOADS_FOLDER = "downloads"
BALANCOS_FOLDER = "balancos"
BALANCOS_XLSX_FOLDER = "balancos_xlsx"
BALANCOS_CONCATENADOS_FOLDER = "balancos_concatenados"
BALANCOS_DEFINITIVO_FOLDER = "balancos_definitivos"

# Criar pastas se não existirem
for folder in [DOWNLOADS_FOLDER, BALANCOS_FOLDER, BALANCOS_XLSX_FOLDER, BALANCOS_CONCATENADOS_FOLDER, BALANCOS_DEFINITIVO_FOLDER]:
    os.makedirs(folder, exist_ok=True)

## Célula 2: Configuração do WebDriver e Extração da Tabela de Resultados


In [ ]:
# Célula 2: Configuração do WebDriver e Extração da Tabela de Resultados
# Configuração do WebDriver em modo headless
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)

# URL da página de resultados
url_resultado = "https://www.fundamentus.com.br/resultado.php"

# Função para obter a tabela HTML
def obter_tabela_html(url, xpath):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    elemento = driver.find_element("xpath", xpath)
    return elemento.get_attribute("outerHTML")

# Obter a tabela HTML
html_tabela = obter_tabela_html(url_resultado, "/html/body/div[1]/div[2]/table")

# Converter a tabela HTML em DataFrame
df_dados_financeiros = pd.read_html(StringIO(html_tabela), decimal=",", thousands=".")[0]
df_dados_financeiros = df_dados_financeiros.sort_values(by="Papel", ascending=True).reset_index(drop=True)

# Exibir o DataFrame
print("Tabela com os dados financeiros:")
display(df_dados_financeiros)

# Lista de empresas
lista_de_empresas = df_dados_financeiros["Papel"].tolist()
print("Quantidade de empresas:", len(lista_de_empresas))

# Fechar o WebDriver
driver.quit()

In [4]:
df_dados_financeiros.to_excel("dados_financeiros.xlsx", index=False)

## Célula 3: Funções Auxiliares

In [7]:
# Célula 3: Funções Auxiliares
def obter_trimestre_anterior():
    mes_atual = datetime.now().month
    ano_atual = datetime.now().year
    if mes_atual <= 3:
        return f"Q4-{ano_atual-1}"
    elif mes_atual <= 6:
        return f"Q1-{ano_atual}"
    elif mes_atual <= 9:
        return f"Q2-{ano_atual}"
    else:
        return f"Q3-{ano_atual}"

trimestre_anterior = obter_trimestre_anterior()
pasta_trimestre = os.path.join(DOWNLOADS_FOLDER, trimestre_anterior)
os.makedirs(pasta_trimestre, exist_ok=True)

## Célula 4: Download dos Arquivos

In [ ]:
# Célula 4: Download dos Arquivos
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": os.path.join(os.getcwd(), pasta_trimestre),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=options)

def baixar_e_renomear_arquivo(papel):
    arquivo_destino = os.path.join(pasta_trimestre, f"{papel}.zip")
    if os.path.isfile(arquivo_destino):
        logging.info(f"Arquivo {arquivo_destino} já existe. Pulando o papel {papel}.")
        return

    url_papeis = f"https://www.fundamentus.com.br/balancos.php?papel={papel}&interface=mobile"
    driver.get(url_papeis)

    try:
        time.sleep(2)
        botao_download = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "a.bt-baixar"))
        )
        botao_download.click()

        # Esperar o download começar
        for _ in range(10):
            time.sleep(5)
            if any(f.startswith("bal_") for f in os.listdir(pasta_trimestre)):
                break

        novo_arquivo = next((f for f in os.listdir(pasta_trimestre) if f.startswith("bal_")), None)

        if novo_arquivo:
            caminho_arquivo = os.path.join(pasta_trimestre, novo_arquivo)
            os.rename(caminho_arquivo, arquivo_destino)
            logging.info(f"Arquivo {arquivo_destino} baixado e renomeado com sucesso!")
        else:
            logging.error(f"Erro ao encontrar o arquivo baixado para {papel}")

    except Exception as e:
        logging.error(f"Erro ao baixar arquivo para {papel}: {e}")

# Baixar e renomear arquivos para cada empresa
for papel in lista_de_empresas:
    baixar_e_renomear_arquivo(papel)

# Fechar o WebDriver
driver.quit()

## Célula 5: Extração e Renomeação dos Arquivos

In [ ]:
# Célula 5: Extração e Renomeação dos Arquivos

# Configuração básica do logging
logging.basicConfig(filename='extracao_renomeacao.log', level=logging.ERROR)

# Função de extração e renomeação
def extrair_e_renomear_arquivo(papel):
    caminho_arquivo_zip = os.path.join(pasta_trimestre, f"{papel}.zip")
    pasta_balancos_trimestre = os.path.join(BALANCOS_FOLDER, trimestre_anterior)
    os.makedirs(pasta_balancos_trimestre, exist_ok=True)
    caminho_arquivo_xls = os.path.join(pasta_balancos_trimestre, "balanco.xls")
    novo_nome_arquivo = os.path.join(pasta_balancos_trimestre, f"{papel}.xls")

    # Verifica se o arquivo já foi extraído e renomeado
    if os.path.exists(novo_nome_arquivo):
        print(f"Arquivo {novo_nome_arquivo} já foi extraído e renomeado.")
        return True  # Indica sucesso sem necessidade de renomeação

    try:
        # Extraindo o arquivo ZIP
        with zipfile.ZipFile(caminho_arquivo_zip, "r") as zip_ref:
            zip_ref.extractall(pasta_balancos_trimestre)

        # Renomeando o arquivo
        if os.path.exists(caminho_arquivo_xls):
            os.rename(caminho_arquivo_xls, novo_nome_arquivo)
            print(f"Arquivo {novo_nome_arquivo} extraído e renomeado com sucesso!")
        else:
            print(f"Arquivo balanco.xls não encontrado em {pasta_balancos_trimestre}")
            return False  # Indica falha na renomeação

    except Exception as e:
        logging.error(f"Erro ao extrair e renomear arquivo para {papel}: {e}")
        return False  # Indica falha na operação

    return True  # Indica sucesso

# Lista para armazenar papéis com erro
papeis_com_erro = []

# Extração e renomeação dos arquivos para cada empresa
for papel in lista_de_empresas[:]:  # Usar uma cópia da lista para evitar problemas ao remover elementos
    sucesso = extrair_e_renomear_arquivo(papel)
    if not sucesso:
        papeis_com_erro.append(papel)

# Remover os papéis com erro da lista de empresas
for papel in papeis_com_erro:
    lista_de_empresas.remove(papel)

# Exibir os papéis que deram erro
print("PAPÉIS COM ERRO:", papeis_com_erro)
print("LISTA DE EMPRESAS ATUALIZADA:", lista_de_empresas)
print("QUANTIDADE DE EMPRESAS ATUALIZADA:", len(lista_de_empresas))


## Célula 6: Conversão de XLS para XLSX

In [ ]:
# Célula 6: Conversão de XLS para XLSX
def converter_xls_para_xlsx(papel):
    pasta_balancos_trimestre = os.path.join(BALANCOS_FOLDER, trimestre_anterior)
    pasta_balancos_xlsx_trimestre = os.path.join(BALANCOS_XLSX_FOLDER, trimestre_anterior)
    os.makedirs(pasta_balancos_xlsx_trimestre, exist_ok=True)

    caminho_xls = os.path.join(pasta_balancos_trimestre, f"{papel}.xls")
    caminho_xlsx = os.path.join(pasta_balancos_xlsx_trimestre, f"{papel}.xlsx")

    if os.path.isfile(caminho_xlsx):
        print(f"Arquivo {papel}.xlsx já foi convertido.")
        return

    try:
        xls = pd.ExcelFile(caminho_xls, engine="xlrd")
        with pd.ExcelWriter(caminho_xlsx, engine="openpyxl") as writer:
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name)
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Arquivo {papel}.xls convertido para {papel}.xlsx.")
    except Exception as e:
        print(f"Erro ao converter {papel}.xls: {e}")

# Converter arquivos XLS para XLSX para cada empresa
for papel in lista_de_empresas:
    converter_xls_para_xlsx(papel)

## Concatenação das Tabelas

In [ ]:
# Configurar logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Constantes
BAL_PATRIM = "Bal. Patrim."
DEM_RESULT = "Dem. Result."
EXTENSAO_EXCEL = ".xlsx"


def reordenar_primeira_linha(df: pd.DataFrame) -> pd.DataFrame:
    """
    Reordena a primeira linha de um DataFrame, colocando as datas da mais antiga para a mais recente.
    """
    # Identificar a primeira linha (após remoção da original) para as datas
    primeira_linha = df.iloc[0, 1:]

    # Tentar converter os valores para datetime, forçando o formato "dd/mm/yyyy"
    datas = pd.to_datetime(primeira_linha, format="%d/%m/%Y", errors="coerce")

    # Verificar se todas as datas são válidas
    if datas.isnull().all():
        logging.warning(
            "Nenhuma data foi reconhecida na primeira linha. Retornando DataFrame original."
        )
        return df

    # Criar um DataFrame temporário para facilitar a reordenação
    temp_df = pd.DataFrame({"original_col": df.columns[1:], "date": datas})

    # Remover entradas que não são datas válidas
    temp_df = temp_df.dropna(subset=["date"])

    # Ordenar as colunas com base nas datas
    temp_df = temp_df.sort_values(by="date")

    # Reordenar o DataFrame original de acordo com as colunas ordenadas
    new_order = [df.columns[0]] + temp_df["original_col"].tolist()
    df = df[new_order]

    # Substituir a primeira linha com as datas ordenadas
    df.iloc[0, 1:] = temp_df["date"].dt.strftime("%d/%m/%Y").values

    return df


def processar_arquivo(caminho_arquivo: str, pasta_saida: str) -> None:
    try:
        nome_arquivo = os.path.basename(caminho_arquivo)
        nome_papel = nome_arquivo.replace(EXTENSAO_EXCEL, "")

        bal_patrim = pd.read_excel(
            caminho_arquivo, sheet_name=BAL_PATRIM, header=None, engine="openpyxl"
        )
        dem_result = pd.read_excel(
            caminho_arquivo, sheet_name=DEM_RESULT, header=None, engine="openpyxl"
        )

        # Remover a primeira linha de ambas as planilhas
        bal_patrim = bal_patrim.iloc[1:].reset_index(drop=True)
        dem_result = dem_result.iloc[1:].reset_index(drop=True)

        # Reordenar a primeira linha de ambas as planilhas
        bal_patrim = reordenar_primeira_linha(bal_patrim)
        dem_result = reordenar_primeira_linha(dem_result)

        # Verificar o conteúdo da célula [0, 0] da planilha 'Bal. Patrim.'
        primeira_celula = bal_patrim.iloc[0, 0]

        if (
            pd.to_datetime(primeira_celula, format="%d/%m/%Y", errors="coerce")
            is not pd.NaT
        ):
            # Se a célula contém uma data, mover as colunas para a direita
            bal_patrim = bal_patrim.shift(1, axis=1)

        # Definir o nome do papel na célula [0, 0] da planilha 'Bal. Patrim.'
        bal_patrim.iloc[0, 0] = nome_papel

        # Remover a primeira linha de 'Dem. Result.' antes de concatenar
        dem_result = dem_result.iloc[1:].reset_index(drop=True)

        # Concatenar as duas planilhas
        combinado = pd.concat([bal_patrim, dem_result], ignore_index=True)

        # Verificação final: Se a célula [0, 0] não contiver o nome do papel, remover a primeira linha
        if combinado.iloc[0, 0] != nome_papel:
            combinado = combinado.iloc[1:].reset_index(drop=True)

        # Criar a pasta de saída correspondente
        nome_subpasta = os.path.basename(os.path.dirname(caminho_arquivo))
        subpasta_saida = os.path.join(pasta_saida, nome_subpasta)
        os.makedirs(subpasta_saida, exist_ok=True)

        # Criar o caminho para o arquivo de saída
        arquivo_saida = os.path.join(subpasta_saida, nome_arquivo)

        # Verificar se o arquivo já existe antes de salvar
        if os.path.exists(arquivo_saida):
            logging.info(f"Arquivo já existe e será ignorado: {arquivo_saida}")
        else:
            # Salvar o resultado em uma nova planilha
            combinado.to_excel(arquivo_saida, index=False)
            logging.info(f"Arquivo combinado e ordenado salvo como: {arquivo_saida}")

    except Exception as e:
        logging.error(f"Erro ao processar o arquivo {caminho_arquivo}: {str(e)}")


def processar_todos_arquivos(pasta_base: str, pasta_saida: str) -> None:
    for nome_subpasta in sorted(os.listdir(pasta_base)):
        caminho_subpasta = os.path.join(pasta_base, nome_subpasta)
        if os.path.isdir(caminho_subpasta):
            for nome_arquivo in sorted(os.listdir(caminho_subpasta)):
                if nome_arquivo.endswith(EXTENSAO_EXCEL):
                    caminho_arquivo = os.path.join(caminho_subpasta, nome_arquivo)
                    processar_arquivo(caminho_arquivo, pasta_saida)


# Execução do processamento
processar_todos_arquivos(BALANCOS_XLSX_FOLDER, BALANCOS_CONCATENADOS_FOLDER)


In [ ]:
# Função para obter as subpastas ordenadas da mais antiga para a mais recente
def obter_subpastas_ordenadas(base_folder):
    subpastas = [f.name for f in os.scandir(base_folder) if f.is_dir()]
    subpastas.sort(
        key=lambda x: datetime.strptime(x.split("-")[1], "%Y")
    )  # Ordenar por ano
    return subpastas

# Função para concatenar as planilhas
def concatenar_planilhas(base_folder, output_folder):
    planilhas_dict = {}

    # Obter subpastas ordenadas
    subpastas_ordenadas = obter_subpastas_ordenadas(base_folder)

    # Percorre todas as subpastas (ordenadas pela data da pasta mais antiga para a mais recente)
    for subpasta in subpastas_ordenadas:
        subpasta_path = os.path.join(base_folder, subpasta)
        if os.path.isdir(subpasta_path):
            for file_name in os.listdir(subpasta_path):
                if file_name.endswith(".xlsx"):
                    file_path = os.path.join(subpasta_path, file_name)

                    # Carregar a planilha, ignorando a primeira linha (índice de colunas)
                    df = pd.read_excel(file_path, skiprows=1)

                    # Se o arquivo já foi processado, concatenar as novas colunas
                    if file_name in planilhas_dict:
                        df_existente = planilhas_dict[file_name]

                        # Ignorar a primeira coluna (indicadores financeiros)
                        for col in df.columns[1:]:
                            if col not in df_existente.columns:
                                df_existente[col] = df[col]

                        # Reordenar as colunas pela data
                        colunas_datas = [
                            col
                            for col in df_existente.columns
                            if col != df_existente.columns[0]
                        ]
                        colunas_datas.sort(
                            key=lambda date: datetime.strptime(str(date), "%d/%m/%Y")
                        )
                        colunas_ordenadas = [df_existente.columns[0]] + colunas_datas
                        planilhas_dict[file_name] = df_existente[colunas_ordenadas]
                    else:
                        # Armazenar a primeira planilha encontrada
                        planilhas_dict[file_name] = df

    # Salvar os arquivos concatenados na pasta de destino
    for nome_arquivo, df_final in planilhas_dict.items():
        # Salvar o DataFrame concatenado no arquivo de destino
        output_path = os.path.join(output_folder, nome_arquivo)
        df_final.to_excel(output_path, index=False)

# Executar a função
concatenar_planilhas(BALANCOS_CONCATENADOS_FOLDER, BALANCOS_DEFINITIVO_FOLDER)


## Dicionarios

### Dicionario de balanços

In [ ]:
# Dicionário para armazenar os DataFrames
dicionario_de_balancos = {}
contador_de_empresas = 0
# Percorre todos os arquivos na pasta 'balancos_definitivos'
for arquivo in os.listdir(BALANCOS_DEFINITIVO_FOLDER):
    if arquivo.endswith(".xlsx"):
        # Cria o caminho completo para o arquivo
        caminho_arquivo = os.path.join(BALANCOS_DEFINITIVO_FOLDER, arquivo)
        contador_de_empresas = contador_de_empresas + 1
        # Carrega o DataFrame a partir do arquivo Excel
        balanco = pd.read_excel(caminho_arquivo, sheet_name=0, engine="openpyxl", header=None)

        # Na célula [0, 0] está o nome da empresa
        nome = balanco.iloc[0, 0]
        print(f"Processando empresa {nome}...")
        # Pega a 1ª linha e torna um cabeçalho
        balanco.columns = balanco.iloc[0]
        balanco = balanco[1:]

        # Define a 1ª coluna (nome da empresa) como índice
        balanco = balanco.set_index(nome)

        # Adiciona o DataFrame ao dicionário com o nome da empresa como chave
        dicionario_de_balancos[nome] = balanco
print("TOTAL DE EMPRESAS: ", contador_de_empresas)

In [ ]:
# Conjunto de nomes no dicionário
nomes_no_dicionario = set(dicionario_de_balancos.keys())

# Conjunto de nomes na lista de empresas
nomes_na_lista = set(lista_de_empresas)

# Empresas que estão no dicionário mas não na lista de empresas
faltando_na_lista = nomes_no_dicionario - nomes_na_lista

# Atualizar lista_de_empresas para incluir as empresas que estão no dicionário
lista_de_empresas.extend(faltando_na_lista)

# Remover duplicatas da lista_de_empresas (caso haja)
lista_de_empresas = list(set(lista_de_empresas))
sorted_lista_de_empresas = sorted(lista_de_empresas)
print("LISTA ATUALIZADA DE EMPRESAS:")
print(sorted_lista_de_empresas)

# Opcional: Exibir as empresas que estavam no dicionário, mas não na lista antes da atualização
print("Empresas que foram adicionadas à lista:")
print(faltando_na_lista)

In [ ]:
# Configurações
data_referencia = "01/01/2023"
data_referencia_aux = pd.to_datetime(data_referencia, format="%d/%m/%Y")

# Lista para armazenar empresas a serem removidas
lista_para_remover = []

# Filtra empresas com último balanço anterior à data de referência
for papel, balanco in dicionario_de_balancos.items():
    colunas = list(balanco.columns)

    try:
        # Converte a última coluna para datetime, especificando o formato e dayfirst=True
        ultima_data_aux = pd.to_datetime(colunas[-1], format="%d/%m/%Y", dayfirst=True)

        if ultima_data_aux <= data_referencia_aux:
            lista_para_remover.append(papel)
    except Exception as e:
        print(f"Erro ao processar a data para {papel}: {e}")

# Exibe a lista de empresas que serão removidas
print("LISTA DE EMPRESAS A SEREM REMOVIDAS: ", lista_para_remover)

# Remove as empresas do dicionário
for papel in lista_para_remover:
    dicionario_de_balancos.pop(papel)

# Exibe o total remanescente e a lista atualizada de empresas
print("\nTOTAL REMANESCENTE:", len(dicionario_de_balancos.keys()))


In [ ]:
dicionario_de_balancos["PETR4"]

### Dicionario de cotações

In [ ]:
# Dicionário para armazenar as cotações
dicionario_de_cotacoes = {}
tickers_falhados = []
keys_list = list(dicionario_de_balancos.keys())

# Itera sobre as chaves do dicionário de balanços
for papel in keys_list:
    # Adiciona ".SA" ao nome do papel para buscar na B3
    papel_com_sufixo = papel + ".SA"
    
    try:
        print(f"Obtendo cotações para {papel_com_sufixo}...")
        
        # Obtém os dados de mercado usando o Yahoo Finance
        cotacao = yf.download(papel_com_sufixo)
        
        if not cotacao.empty:
            # Seleciona as colunas desejadas e organiza o DataFrame
            cotacao_filtrada = cotacao[['Open', 'High', 'Low', 'Close', 'Adj Close']]
            cotacao_filtrada.columns = ['Abertura', 'Máxima', 'Baixa', 'Fechamento', 'Fechamento Ajustado']
            
            # Transpõe o DataFrame para que as colunas virem linhas e as datas sejam colunas
            cotacao_transposta = cotacao_filtrada.transpose()
            
            # Adiciona o DataFrame transposto ao dicionário de cotações
            dicionario_de_cotacoes[papel] = cotacao_transposta
        else:
            print(f"Cotações não encontradas para {papel_com_sufixo}")
            tickers_falhados.append(papel_com_sufixo)

    except Exception as e:
        print(f"Erro ao baixar cotações para {papel_com_sufixo}: {e}")
        tickers_falhados.append(papel_com_sufixo)

# Exibe o resultado
print("\nTOTAL DE COTAÇÕES OBTIDAS:", len(dicionario_de_cotacoes.keys()))
print("\nCOTAÇÕES OBTIDAS PARA OS SEGUINTES TICKERS:", dicionario_de_cotacoes.keys())
print("COTAÇÕES NÃO ENCONTRADAS PARA OS SEGUINTES TICKERS:", tickers_falhados)

In [ ]:
# Configurações
data_referencia = "2023/01/01"
data_referencia_aux = pd.to_datetime(data_referencia)

# Lista para armazenar empresas a serem removidas
lista_para_remover = []

# Filtra empresas com última cotação anterior à data de referência
for papel, cotacao in dicionario_de_cotacoes.items():
    try:
        # Extraindo as datas das colunas do DataFrame transposto
        datas = cotacao.columns

        # Verifica se há datas disponíveis
        if not datas.empty:
            # Converte a última data para datetime
            ultima_data_aux = pd.to_datetime(datas[-1])

            # Compara a última data com a data de referência
            if ultima_data_aux <= data_referencia_aux:
                lista_para_remover.append(papel)
        else:
            print(f"Sem dados de datas para {papel}.")
            lista_para_remover.append(papel)

    except Exception as e:
        print(f"Erro ao processar a data para {papel}: {e}")
        lista_para_remover.append(papel)

# Exibe a lista de empresas que serão removidas
print("LISTA DE EMPRESAS A SEREM REMOVIDAS: ", lista_para_remover)

# Remove as empresas do dicionário
for papel in lista_para_remover:
    dicionario_de_cotacoes.pop(papel, None)

# Exibe o total remanescente e a lista atualizada de empresas
print("\nTOTAL REMANESCENTE:", len(dicionario_de_cotacoes.keys()))
print("\nEMPRESAS REMANESCENTES:", list(dicionario_de_cotacoes.keys()))

In [ ]:
dicionario_de_cotacoes["AALR3"]

### Chaves comuns entre os dicionários

In [ ]:
# Obtém as chaves de ambos os dicionários
chaves_balancos = set(dicionario_de_balancos.keys())
chaves_cotacoes = set(dicionario_de_cotacoes.keys())

# Encontra as chaves comuns
chaves_comuns = chaves_balancos.intersection(chaves_cotacoes)

# Remove as entradas dos dicionários que não estão nas chaves comuns
for chave in list(dicionario_de_balancos.keys()):
    if chave not in chaves_comuns:
        del dicionario_de_balancos[chave]

for chave in list(dicionario_de_cotacoes.keys()):
    if chave not in chaves_comuns:
        del dicionario_de_cotacoes[chave]

print("Dicionários atualizados!")
print(f"Total de empresas restantes nos balanços: {len(dicionario_de_balancos.keys())}")
print(f"Total de empresas restantes nas cotações: {len(dicionario_de_cotacoes.keys())}")

## Graham e PEG

### Graham

In [ ]:
# Função para calcular o valor de Graham
def calcular_graham(lpa, vpa):
    return math.sqrt(22.5 * lpa * vpa) if lpa > 0 and vpa > 0 else None


# Função para pegar o número de ações do Yahoo Finance
def obter_numero_acoes(papel):
    ticker = yf.Ticker(f"{papel}.SA")
    numero_acoes = ticker.info.get("sharesOutstanding", None)  # Número de ações
    return numero_acoes


# Transformar as chaves do dicionário em uma lista
lista_empresas = list(dicionario_de_balancos.keys())

# Dicionário para armazenar os DataFrames de Graham
dicionario_de_graham = {}
contador_de_empresas = 0

# Lista para armazenar empresas com erro
empresas_com_erro = []

# Percorre a lista de empresas no dicionário
for empresa in lista_empresas:
    arquivo = f"{empresa}.xlsx"  # Formato esperado dos arquivos na pasta
    caminho_arquivo = os.path.join(BALANCOS_DEFINITIVO_FOLDER, arquivo)

    # Verifica se o arquivo existe
    if os.path.exists(caminho_arquivo):
        try:
            # Carrega o DataFrame a partir do arquivo Excel
            balanco = pd.read_excel(
                caminho_arquivo, sheet_name=0, engine="openpyxl", header=None
            )

            # Na célula [0, 0] está o nome da empresa (mesmo que já saibamos da lista)
            nome = balanco.iloc[0, 0]
            print(f"Processando empresa {nome}...")

            # Pega a 1ª linha e torna um cabeçalho
            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]

            # Define a 1ª coluna (nome da empresa) como índice
            balanco = balanco.set_index(balanco.columns[0])

            # Pega o número de ações da empresa
            numero_acoes = obter_numero_acoes(nome)

            if numero_acoes:
                # Inicializa dicionários para armazenar LPA, VPA e Graham
                lpa_dict = {}
                vpa_dict = {}
                graham_dict = {}

                # Itera sobre as colunas de datas
                for coluna in balanco.columns[1:]:
                    try:
                        # Pega o Patrimônio Líquido e o Lucro Líquido para a data
                        patrimonio_liquido = (
                            balanco.loc["Patrimônio Líquido", coluna] * 1000
                        )
                        lucro_liquido = (
                            balanco.loc["Lucro/Prejuízo do Período", coluna] * 1000
                        )

                        # Calcula o LPA e o VPA
                        lpa = lucro_liquido / numero_acoes
                        vpa = patrimonio_liquido / numero_acoes

                        # Calcula o valor de Graham
                        valor_graham = calcular_graham(lpa, vpa)

                        # Armazena LPA, VPA e Graham em seus respectivos dicionários
                        lpa_dict[coluna] = lpa
                        vpa_dict[coluna] = vpa
                        graham_dict[coluna] = valor_graham

                        contador_de_empresas += 1

                    except Exception as e:
                        print(f"Erro ao calcular Graham para {nome} na data {coluna}: {e}")
                        empresas_com_erro.append(nome)

                # Cria o DataFrame com LPA, VPA e Graham como índice e datas como colunas
                df_graham = pd.DataFrame(
                    {"LPA": lpa_dict, "VPA": vpa_dict, "Valor de Graham": graham_dict}
                ).T  # Transpõe o DataFrame para que as datas sejam colunas

                # Adiciona o DataFrame ao dicionário principal
                dicionario_de_graham[nome] = df_graham

            else:
                print(f"Erro ao obter número de ações para {nome}")
                empresas_com_erro.append(nome)

        except Exception as e:
            print(f"Erro ao processar a empresa {empresa}: {e}")
            empresas_com_erro.append(nome)

    else:
        print(f"Arquivo {arquivo} não encontrado.")
        empresas_com_erro.append(empresa)

# Exibe o total de empresas processadas
print("TOTAL DE EMPRESAS: ", contador_de_empresas)
print(f"TOTAL DE EMPRESAS NO DICIONÁRIO DE GRAHAM: {len(dicionario_de_graham.keys())}")

# Exibe a lista de empresas que deram erro
if empresas_com_erro:
    print("Empresas que apresentaram erro:", empresas_com_erro)
else:
    print("Nenhuma empresa apresentou erro.")

In [ ]:
dicionario_de_graham["PETR4"]

### PEG

### PEG com Erro

In [ ]:
# Função para calcular o PEG Ratio
def calcular_peg(preco, lpa, crescimento_lucro):
    try:
        # Evita divisões por zero ou valores negativos
        if isinstance(lpa, (int, float)) and isinstance(crescimento_lucro, (int, float)) and lpa > 0 and crescimento_lucro > 0:
            return (preco / lpa) / crescimento_lucro
        else:
            return None
    except Exception as e:
        print(f"Erro ao calcular PEG: {e}")
        return None

# Função para converter valores em float, lidando com possíveis erros
def converter_para_float(valor):
    try:
        return float(valor)
    except ValueError:
        return None

# Função para pegar o preço de fechamento do dicionário de cotações
def obter_preco_fechamento(papel, data):
    try:
        # Verifica se o papel está no dicionário de cotações
        if papel in dicionario_de_cotacoes:
            cotacoes = dicionario_de_cotacoes[papel]
            data_formatada = pd.to_datetime(data, format='%d/%m/%Y', errors='coerce')
            
            if data in cotacoes.columns:
                return float(cotacoes.loc['Fechamento', data])
            else:
                datas_disponiveis = pd.to_datetime(cotacoes.columns, errors='coerce')
                datas_disponiveis = datas_disponiveis[datas_disponiveis <= data_formatada]
                
                if not datas_disponiveis.empty:
                    ultima_data = datas_disponiveis[-1]
                    return float(cotacoes.loc['Fechamento', str(ultima_data.date())])
                else:
                    print(f"Nenhuma data anterior encontrada para {papel}.")
                    return None
        else:
            print(f"Cotações não encontradas para {papel}.")
            return None
    except Exception as e:
        print(f"Erro ao obter preço de fechamento para {papel} na data {data}: {e}")
        return None

# Transformar as chaves do dicionário em uma lista
sorted_lista_de_empresas = ["AALR3"]

# Dicionário para armazenar os DataFrames de PEG
dicionario_de_peg = {}
contador_de_empresas = 0

# Lista para armazenar empresas com erro
empresas_com_erro = []

# Percorre a lista de empresas no dicionário
for empresa in sorted_lista_de_empresas:
    arquivo = f"{empresa}.xlsx"
    caminho_arquivo = os.path.join(BALANCOS_DEFINITIVO_FOLDER, arquivo)

    if os.path.exists(caminho_arquivo):
        try:
            balanco = pd.read_excel(caminho_arquivo, sheet_name=0, engine="openpyxl", header=None)

            nome = balanco.iloc[0, 0]
            print(f"Processando empresa {nome}...")

            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]
            balanco = balanco.set_index(balanco.columns[0])

            lpa_dict = {}
            crescimento_dict = {}
            peg_dict = {}

            erro_na_empresa = False  # Flag para evitar erros repetidos

            # Itera sobre as colunas de datas
            for coluna in balanco.columns[1:]:
                try:
                    lucro_liquido = converter_para_float(balanco.loc["Lucro/Prejuízo do Período", coluna]) * 1000
                    lucro_liquido_anterior = converter_para_float(balanco.loc["Lucro/Prejuízo do Período", balanco.columns[balanco.columns.get_loc(coluna) - 1]]) * 1000 if coluna > 1 else None
                    
                    if lucro_liquido is None or (lucro_liquido_anterior is not None and lucro_liquido_anterior <= 0):
                        erro_na_empresa = True
                        break

                    numero_acoes = obter_numero_acoes(nome)
                    if not numero_acoes:
                        erro_na_empresa = True
                        break

                    lpa = lucro_liquido / numero_acoes
                    crescimento_lucro = ((lucro_liquido - lucro_liquido_anterior) / abs(lucro_liquido_anterior)) if lucro_liquido_anterior else None

                    preco_acao = obter_preco_fechamento(empresa, coluna)
                    if preco_acao is None:
                        erro_na_empresa = True
                        break

                    valor_peg = calcular_peg(preco_acao, lpa, crescimento_lucro)

                    lpa_dict[coluna] = lpa
                    crescimento_dict[coluna] = crescimento_lucro
                    peg_dict[coluna] = valor_peg

                except Exception as e:
                    print(f"Erro ao calcular PEG para {nome} na data {coluna}: {e}")
                    erro_na_empresa = True
                    break

            if erro_na_empresa:
                empresas_com_erro.append(nome)
            else:
                df_peg = pd.DataFrame({"LPA": lpa_dict, "Crescimento do Lucro": crescimento_dict, "PEG": peg_dict}).T
                dicionario_de_peg[nome] = df_peg
                contador_de_empresas += 1

        except Exception as e:
            print(f"Erro ao processar a empresa {empresa}: {e}")
            empresas_com_erro.append(nome)

    else:
        print(f"Arquivo {arquivo} não encontrado.")
        empresas_com_erro.append(empresa)

# Exibe o total de empresas processadas e a lista de empresas com erro
print(f"TOTAL DE EMPRESAS NO DICIONÁRIO DE PEG: {len(dicionario_de_peg.keys())}")
if empresas_com_erro:
    print("Empresas que apresentaram erro:", set(empresas_com_erro))  # Usa set para evitar duplicados
else:
    print("Nenhuma empresa apresentou erro.")

In [ ]:
dicionario_de_peg["AALR3"]